# Audio Similarity Search

## Setup

In [ ]:
from datasets import load_dataset

--2025-04-22 10:05:23--  http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv
Resolvendo storage.googleapis.com (storage.googleapis.com)... 2800:3f0:4001:83b::201b, 2800:3f0:4001:839::201b, 2800:3f0:4001:838::201b, ...
Conectando-se a storage.googleapis.com (storage.googleapis.com)|2800:3f0:4001:83b::201b|:80... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 14675 (14K) [application/octet-stream]
Salvando em: “/Users/viniciusfinger/panns_data/class_labels_indices.csv”

     0K .......... ....                                       100%  368K=0,04s

2025-04-22 10:05:23 (368 KB/s) - “/Users/viniciusfinger/panns_data/class_labels_indices.csv” salvo [14675/14675]



## Load Data

In [2]:
data = load_dataset("ashraq/esc50", split="train")

README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

(…)-00000-of-00002-2f1ab7b824ec751f.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

(…)-00001-of-00002-27425e5c1846b494.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
data

Dataset({
    features: ['filename', 'fold', 'target', 'category', 'esc10', 'src_file', 'take', 'audio'],
    num_rows: 2000
})

In [4]:
data["category"]

['dog',
 'chirping_birds',
 'vacuum_cleaner',
 'vacuum_cleaner',
 'thunderstorm',
 'thunderstorm',
 'door_wood_knock',
 'can_opening',
 'crow',
 'door_wood_knock',
 'door_wood_knock',
 'clapping',
 'clapping',
 'clapping',
 'dog',
 'clapping',
 'thunderstorm',
 'fireworks',
 'fireworks',
 'fireworks',
 'fireworks',
 'clapping',
 'clapping',
 'clapping',
 'chainsaw',
 'airplane',
 'mouse_click',
 'pouring_water',
 'train',
 'sheep',
 'water_drops',
 'water_drops',
 'water_drops',
 'church_bells',
 'church_bells',
 'clock_alarm',
 'keyboard_typing',
 'wind',
 'clock_alarm',
 'footsteps',
 'footsteps',
 'footsteps',
 'footsteps',
 'footsteps',
 'footsteps',
 'frog',
 'frog',
 'fireworks',
 'fireworks',
 'cow',
 'water_drops',
 'brushing_teeth',
 'brushing_teeth',
 'car_horn',
 'crackling_fire',
 'helicopter',
 'helicopter',
 'helicopter',
 'helicopter',
 'helicopter',
 'helicopter',
 'drinking_sipping',
 'rain',
 'crackling_fire',
 'insects',
 'crackling_fire',
 'crackling_fire',
 'crackl

In [6]:
len(data["category"]) # 50 different categories, 40 samples per category (50 * 40)

2000

## Displaying Audio

In [7]:
import numpy as np

audios = np.array([file["array"] for file in data["audio"]])

audios

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.18408203e-02, -1.03363037e-01, -1.41418457e-01, ...,
         6.98547363e-02,  4.04968262e-02,  2.74658203e-03],
       [-6.95800781e-03, -1.25122070e-02, -1.12609863e-02, ...,
         2.15270996e-01, -8.75854492e-03, -2.89031982e-01],
       ...,
       [-8.87145996e-02, -7.35473633e-02, -6.95800781e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-5.01800537e-01, -2.22229004e-01,  5.27648926e-02, ...,
         6.01501465e-02,  6.01196289e-02,  5.78613281e-02],
       [-3.96728516e-04, -1.22070312e-04, -4.57763672e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [16]:
from IPython.display import Audio, display

sound_num = 1999

sound_array = data[sound_num]["audio"]["array"]
sr = data[sound_num]["audio"]["sampling_rate"]
print(f"Class: {data[sound_num]['category']}")
display(Audio(sound_array, rate=sr))

Class: dog


## Audio Model Setup

In [17]:
import torch
from panns_inference import AudioTagging

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AudioTagging(device=device, checkpoint_path=None)


Checkpoint path: /Users/viniciusfinger/panns_data/Cnn14_mAP=0.431.pth


--2025-04-22 11:34:26--  https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth?download=1
Resolvendo zenodo.org (zenodo.org)... 2001:1458:d00:61::100:2f3, 2001:1458:d00:24::100:f6, 2001:1458:d00:25::100:372, ...
Conectando-se a zenodo.org (zenodo.org)|2001:1458:d00:61::100:2f3|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 301 MOVED PERMANENTLY
Localização: /records/3987831/files/Cnn14_mAP=0.431.pth [redirecionando]
--2025-04-22 11:34:27--  https://zenodo.org/records/3987831/files/Cnn14_mAP=0.431.pth
Reaproveitando a conexão existente para [zenodo.org]:443.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 327428481 (312M) [application/octet-stream]
Salvando em: “/Users/viniciusfinger/panns_data/Cnn14_mAP=0.431.pth”

     0K .......... .......... .......... .......... ..........  0%  112K 47m29s
    50K .......... .......... .......... .......... ..........  0% 77,6M 23m46s
   100K .......... .......... .......... .......... .......

Using CPU.


In [24]:
_, sound_tagged = model.inference(sound_array[None, :])

array_dimension = sound_tagged.shape[1]
array_dimension

2048

## Setup Pinecone

In [26]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

idx_name = "audio"

spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1"
)

pc.create_index(
    idx_name,
    dimension=array_dimension,
    metric="cosine",
    spec=spec
)


{
    "name": "audio",
    "metric": "cosine",
    "host": "audio-rdf9dfs.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 2048,
    "deletion_protection": "disabled",
    "tags": null
}

In [27]:
idx = pc.Index(idx_name)

## Vector Upserting

In [31]:
from tqdm.auto import tqdm

batch_size = 64

for i in tqdm(range(0, len(audios), batch_size)):
    i_end = min(len(audios), i + batch_size)
    
    audios_batch = audios[i:i_end]
    
    # Reshape to 2048
    _, embedded_audios = model.inference(audios_batch)
    
    ids = [f"{id}" for id in range(i, i_end)]
    
    metadata = [{"category": category} for category in data[i:i_end]["category"]]
    
    vectors = list(zip(ids, embedded_audios.tolist(), metadata))
    
    idx.upsert(vectors=vectors)
    

  0%|          | 0/32 [00:00<?, ?it/s]